<center>
<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

<center>
Auteur: [Dmitriy Sergeyev](https://github.com/DmitrySerg), Data Scientist @Zeptolab, chargé de cours au Center of Mathematical Finance de MSU.  
    Traduit par: @borowis et [Ousmane Cissé](https://fr.linkedin.com/in/ousmane-cisse).  
Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).  
L'utilisation gratuite est autorisée à des fins non commerciales.


# <center> Analyse des séries temporelles en Python</center>

Bonjour à tous !

Voyons comment travailler avec les séries chronologiques en Python: quelles méthodes et quels modèles pouvons-nous utiliser pour la prédiction, ce qu'est le lissage exponentiel double et triple, que faire si la stationnarité n'est pas votre chose préférée, comment construire SARIMA et rester en vie , comment faire des prédictions en utilisant xgboost ... De plus, tout cela sera appliqué à des exemples (difficiles) du monde réel.

# Plan de l'article:
1. [Introduction](#Introduction)
   - [Forecast quality metrics](#Forecast-quality-metrics)
2. [Move, smoothe, evaluate](#Move,-smoothe,-evaluate)
   - Rolling window estimations
   - Exponential smoothing, Holt-Winters model
   - Time-series cross validation, parameters selection
3. [Econometric approach](#Econometric-approach)
   - Stationarity, unit root
   - Getting rid of non-stationarity
   - SARIMA intuition and model building
4. [Linear (and not quite) models for time series](#Linear-(and-not-quite)-models-for-time-series)
   - [Feature extraction](#Feature-extraction)
   - [Time series lags](#Time-series-lags)
   - [Target encoding](#Target-encoding)
   - [Regularization and feature selection](#Regularization-and-feature-selection)
   - [Boosting](#Boosting)
5. [Conclusion](#Conclusion)
6. [Demo assignment](#Demo-assignment)
7. [Useful resources](#Useful-resources)

Dans mon travail quotidien, je rencontre des tâches liées à des séries chronologiques presque tous les jours. Les questions les plus fréquemment posées sont les suivantes: que se passera-t-il avec nos mesures le lendemain / la semaine / le mois / etc., combien d'utilisateurs installeront notre application, combien de temps ils passeront en ligne, combien d'actions les utilisateurs effectueront-ils, etc. Nous pouvons aborder ces tâches de prédiction en utilisant différentes méthodes en fonction de la qualité requise de la prédiction, de la durée de la période de prévision et, bien sûr, du délai dans lequel nous devons choisir les caractéristiques et régler les paramètres pour obtenir les résultats souhaités.

# Introduction

Nous commençons par une simple [définition](https://en.wikipedia.org/wiki/Time_series) des séries chronologiques:
> *Séries temporelles* est une série de points de données indexés (ou répertoriés ou représentés graphiquement) dans un ordre chronologique.

Par conséquent, les données sont organisées par des horodatages relativement déterministes et peuvent, par rapport aux données d'échantillonnage aléatoire, contenir des informations supplémentaires que nous pouvons extraire.

Importons quelques bibliothèques. Tout d'abord, nous aurons besoin de la bibliothèque [statsmodels](http://statsmodels.sourceforge.net/stable/), qui possède de nombreuses fonctions de modélisation statistique, y compris des séries chronologiques. Pour les afficionados de R qui ont dû passer à Python, `statsmodels` vous sera certainement plus familier car il prend en charge les définitions de modèles   
comme «Salaire ~ Âge + Éducation».

In [ ]:
import matplotlib.pyplot as plt  # plots
import numpy as np  # vectors and matrices
import pandas as pd  # tables and data manipulations
import seaborn as sns  # more plots

sns.set()

import warnings  # `do not disturbe` mode
from itertools import product  # some useful functions

import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.formula.api as smf  # statistics and econometrics
import statsmodels.tsa.api as smt
from dateutil.relativedelta import relativedelta  # working with dates with style
from scipy.optimize import minimize  # for function minimization
from tqdm import tqdm_notebook

warnings.filterwarnings("ignore")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

À titre d'exemple, regardons les vraies données de jeux mobiles. Plus précisément, nous examinerons les annonces regardées par heure et les dépenses en devises dans le jeu par jour:

In [ ]:
ads = pd.read_csv("../../data/ads.csv", index_col=["Time"], parse_dates=["Time"])
currency = pd.read_csv(
    "../../data/currency.csv", index_col=["Time"], parse_dates=["Time"]
)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ads.Ads)
plt.title("Ads watched (hourly data)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(currency.GEMS_GEMS_SPENT)
plt.title("In-game currency spent (daily data)")
plt.grid(True)
plt.show()

## Mesures de qualité des prévisions

Avant de commencer les prévisions, comprenons comment mesurer la qualité de nos prévisions et jetons un œil aux mesures les plus couramment utilisées.

- [R squared](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination): coefficient de détermination (en économétrie, cela peut être interprété comme le pourcentage de variance expliqué par le modèle), $(-\infty, 1]$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$

```python
sklearn.metrics.r2_score
```
---
- [Mean Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error): il s'agit d'une métrique interprétable car elle a la même unité de mesure que la série initiale, $[0, +\infty)$

$MAE = \frac{\sum\limits_{i=1}^{n} |y_i - \hat{y}_i|}{n}$

```python
sklearn.metrics.mean_absolute_error
```
---
                        
- [Median Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error): encore une fois, une métrique interprétable qui est particulièrement intéressante car elle est robuste aux valeurs aberrantes, $[0, +\infty)$

$MedAE = median(|y_1 - \hat{y}_1|, ... , |y_n - \hat{y}_n|)$

```python
sklearn.metrics.median_absolute_error

```
---

- [Mean Squared Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error): la métrique la plus couramment utilisée qui donne une pénalité plus élevée aux grosses erreurs et vice versa, $[0, +\infty)$

$MSE = \frac{1}{n}\sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2$

```python
sklearn.metrics.mean_squared_error
```
---

- [Mean Squared Logarithmic Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-logarithmic-error): pratiquement, la même chose que MSE, mais nous prenez le logarithme de la série. En conséquence, nous accordons également plus de poids aux petites erreurs. Ceci est généralement utilisé lorsque les données ont des tendances exponentielles, $[0, +\infty)$

$MSLE = \frac{1}{n}\sum\limits_{i=1}^{n} (log(1+y_i) - log(1+\hat{y}_i))^2$

```python
sklearn.metrics.mean_squared_log_error
```
---

- Mean Absolute Percentage Error: c'est la même chose que MAE mais est calculé en pourcentage, ce qui est très pratique lorsque vous voulez expliquer la qualité du modèle à la direction, $[0, +\infty)$

$MAPE = \frac{100}{n}\sum\limits_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{y_i}$

```python
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
```
</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
# Importing everything from above

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_squared_log_error,
    median_absolute_error,
    r2_score,
)


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Maintenant que nous savons comment mesurer la qualité des prévisions, voyons quelles mesures nous pouvons utiliser et comment traduire les résultats pour le Directeur. Après cela, un petit détail reste - la construction du modèle.

# Déplacer, lisser, évaluer

Commençons par une hypothèse naïve: "demain sera le même qu'aujourd'hui". Cependant, au lieu d'un modèle comme $\hat{y}_{t} = y_{t-1}$ (qui est en fait une excellente base de référence pour tout problème de prédiction de séries chronologiques et parfois impossible à battre), nous supposerons que la valeur future de notre variable dépend de la moyenne de ses valeurs précédentes $k$. Par conséquent, nous utiliserons la **moyenne mobile**.

$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k}_{n=1} y_{t-n}$

In [ ]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])


moving_average(ads, 24)  # prediction for the last observed day (past 24 hours)

Malheureusement, nous ne pouvons pas faire de prédictions dans le futur - pour obtenir la valeur de l'étape suivante, nous avons besoin que les valeurs précédentes soient réellement observées. Mais la moyenne mobile a un autre cas d'utilisation - lisser la série chronologique d'origine pour identifier les tendances. Pandas a une implémentation disponible avec [`DataFrame.rolling (window).mean ()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). Plus la fenêtre est large, plus la tendance est fluide. Dans le cas de données très bruyantes, que l'on rencontre souvent en finance, cette procédure peut aider à détecter des schémas (patterns) courants.

In [ ]:
def plotMovingAverage(
    series, window, plot_intervals=False, scale=1.96, plot_anomalies=False
):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 

    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15, 5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")

        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series < lower_bond] = series[series < lower_bond]
            anomalies[series > upper_bond] = series[series > upper_bond]
            plt.plot(anomalies, "ro", markersize=10)

    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

Lissons les 4 dernières heures.

In [ ]:
plotMovingAverage(ads, 4)

Essayons maintenant le lissage des 12 dernières heures.

In [ ]:
plotMovingAverage(ads, 12)

Maintenant en lissant les dernières 24 heures, nous obtenons la tendance quotidienne.

In [ ]:
plotMovingAverage(ads, 24)

Lorsque nous avons appliqué un lissage quotidien sur les données horaires, nous avons pu voir clairement la dynamique des annonces vues. Pendant les week-ends, les valeurs sont plus élevées (plus de temps pour jouer le week-end) tandis que moins d'annonces sont regardées en semaine.

Nous pouvons également tracer des intervalles de confiance pour nos valeurs lissées.

In [ ]:
plotMovingAverage(ads, 4, plot_intervals=True)

Maintenant, créons un système de détection d'anomalies simple à l'aide de la moyenne mobile. Malheureusement, dans cet ensemble de données particulier, tout est plus ou moins normal, nous allons donc intentionnellement créer des valeurs anormales dans notre dataframe `ads_anomaly`.

In [ ]:
ads_anomaly = ads.copy()
ads_anomaly.iloc[-20] = ads_anomaly.iloc[-20] * 0.2  # say we have 80% drop of ads

Voyons si cette méthode simple peut détecter l'anomalie.

In [ ]:
plotMovingAverage(ads_anomaly, 4, plot_intervals=True, plot_anomalies=True)

Neat! What about the second series?

Et la deuxième série?

In [ ]:
plotMovingAverage(
    currency, 7, plot_intervals=True, plot_anomalies=True
)  # weekly smoothing

Oh non, ce n'était pas aussi génial! Ici, nous pouvons voir l'inconvénient de notre approche simple - elle n'a pas saisi la saisonnalité mensuelle dans nos données et a marqué presque tous les pics de 30 jours comme des anomalies. Si vous voulez éviter les faux positifs, il est préférable d'envisager des modèles plus complexes.

**Moyenne pondérée** (Weighted average) est une simple modification de la moyenne mobile. Les poids totalisent `1` avec des poids plus importants attribués à des observations plus récentes.


$\hat{y}_{t} = \displaystyle\sum^{k}_{n=1} \omega_n y_{t+1-n}$

In [ ]:
def weighted_average(series, weights):
    """
        Calculate weighted average on the series.
        Assuming weights are sorted in descending order
        (larger weights are assigned to more recent observations).
    """
    result = 0.0
    for n in range(len(weights)):
        result += series.iloc[-n - 1] * weights[n]
    return float(result)

In [ ]:
weighted_average(ads, [0.6, 0.3, 0.1])

In [ ]:
# just checking
0.6 * ads.iloc[-1] + 0.3 * ads.iloc[-2] + 0.1 * ads.iloc[-3]

## Lissage exponentiel

Voyons maintenant ce qui se passe si, au lieu de pondérer les dernières valeurs $k$ de la série chronologique, nous commençons à pondérer toutes les observations disponibles tout en diminuant de façon exponentielle les poids à mesure que nous remontons dans le temps. Il existe une formule pour **[lissage exponentiel](https://en.wikipedia.org/wiki/Exponential_smoothing)** qui nous aidera avec ceci:

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $$

Ici, la valeur du modèle est une moyenne pondérée entre la valeur réelle actuelle et les valeurs précédentes du modèle. Le poids $\alpha$ est appelé facteur de lissage. Il définit la vitesse à laquelle nous "oublierons" la dernière observation vraie disponible. Plus $\alpha$ est petit, plus les observations précédentes ont d'influence et plus la série est fluide.

L'exponentialité est cachée dans la récursivité de la fonction - nous multiplions par $(1-\alpha)$ à chaque fois, qui contient déjà une multiplication par $(1-\alpha)$ des valeurs de modèle précédentes.

In [ ]:
def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]]  # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n - 1])
    return result

In [ ]:
def plotExponentialSmoothing(series, alphas):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context("seaborn-white"):
        plt.figure(figsize=(15, 7))
        for alpha in alphas:
            plt.plot(
                exponential_smoothing(series, alpha), label="Alpha {}".format(alpha)
            )
        plt.plot(series.values, "c", label="Actual")
        plt.legend(loc="best")
        plt.axis("tight")
        plt.title("Exponential Smoothing")
        plt.grid(True);

In [ ]:
plotExponentialSmoothing(ads.Ads, [0.3, 0.05])

In [ ]:
plotExponentialSmoothing(currency.GEMS_GEMS_SPENT, [0.3, 0.05])

## Double lissage exponentiel

Jusqu'à présent, les méthodes dont nous avons discuté, ont porté pour une seule prédiction de points futurs (avec un bon lissage). C'est cool, mais ce n'est pas suffisant non plus. Étendons le lissage exponentiel afin de pouvoir prédire deux points futurs (bien sûr, nous inclurons également plus de lissage).

La décomposition en séries nous aidera - nous obtenons deux composantes: $\ell$ d'interception (c'est-à-dire de niveau) et $b$ de pente (c'est-à-dire de tendance). Nous avons appris à prédire l'interception (ou la valeur de série attendue) avec nos méthodes précédentes; maintenant, nous appliquerons le même lissage exponentiel à la tendance en supposant que la direction future des changements de séries chronologiques dépend des changements pondérés précédents. En conséquence, nous obtenons l'ensemble de fonctions suivant:

$$\ell_x = \alpha y_x + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$\hat{y}_{x+1} = \ell_x + b_x$$

La première décrit l'ordonnée à l'origine, qui, comme précédemment, dépend de la valeur actuelle de la série. Le deuxième terme est maintenant divisé en valeurs précédentes du niveau et de la tendance. La deuxième fonction décrit la tendance, qui dépend des changements de niveau à l'étape actuelle et de la valeur précédente de la tendance. Dans ce cas, le coefficient $\beta$ est un poids pour le lissage exponentiel. La prédiction finale est la somme des valeurs du modèle de l'interception et de la tendance.

In [ ]:
def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series) + 1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series):  # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result


def plotDoubleExponentialSmoothing(series, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """

    with plt.style.context("seaborn-white"):
        plt.figure(figsize=(20, 8))
        for alpha in alphas:
            for beta in betas:
                plt.plot(
                    double_exponential_smoothing(series, alpha, beta),
                    label="Alpha {}, beta {}".format(alpha, beta),
                )
        plt.plot(series.values, label="Actual")
        plt.legend(loc="best")
        plt.axis("tight")
        plt.title("Double Exponential Smoothing")
        plt.grid(True)

In [ ]:
plotDoubleExponentialSmoothing(ads.Ads, alphas=[0.9, 0.02], betas=[0.9, 0.02])

In [ ]:
plotDoubleExponentialSmoothing(
    currency.GEMS_GEMS_SPENT, alphas=[0.9, 0.02], betas=[0.9, 0.02]
)

Maintenant, nous devons régler deux paramètres: $\alpha$ et $\beta$. Le premier est responsable du lissage de la série autour de la tendance, le second du lissage de la tendance elle-même. Plus les valeurs sont élevées, plus les observations les plus récentes auront de poids et moins la série de modèles sera lissée. Certaines combinaisons de paramètres peuvent produire des résultats étranges, surtout si elles sont définies manuellement. Nous nous pencherons dans un instant sur le choix automatique des paramètres ; avant cela, parlons du lissage triple exponentiel.

## Triple lissage exponentiel alias Holt-Winters

Nous avons examiné le lissage exponentiel et le double lissage exponentiel. Cette fois, nous allons passer au _triple_ lissage exponentiel.

Comme vous auriez pu le deviner, l'idée est d'ajouter une troisième composante - la saisonnalité. Cela signifie que nous ne devrions pas utiliser cette méthode si notre série chronologique ne devrait pas avoir de saisonnalité. Les composantes saisonnières du modèle expliqueront les variations répétées autour de l'interception et de la tendance, et elles seront spécifiées par la durée de la saison, en d'autres termes par la période après laquelle les variations se répètent. Pour chaque observation de la saison, il y a une composante distincte; par exemple, si la durée de la saison est de 7 jours (une saisonnalité hebdomadaire), nous aurons 7 composantes saisonnières, une pour chaque jour de la semaine.

Avec cela, écrivons un nouveau système d'équations:

$$\ell_x = \alpha(y_x - s_{x-L}) + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$s_x = \gamma(y_x - \ell_x) + (1-\gamma)s_{x-L}$$

$$\hat{y}_{x+m} = \ell_x + mb_x + s_{x-L+1+(m-1)modL}$$

L'interception dépend maintenant de la valeur actuelle de la série moins toute composante saisonnière correspondante. La tendance reste inchangée et la composante saisonnière dépend de la valeur actuelle de la série moins l'ordonnée à l'origine et de la valeur précédente de la composante. Prendre en compte que la composante est lissée à travers toutes les saisons disponibles; par exemple, si nous avons une composante lundi, elle ne sera calculée qu'en moyenne avec les autres lundis. Vous pouvez en savoir plus sur le fonctionnement de la moyenne et comment se fait l'approximation initiale de la tendance et des composantes saisonnières [ici](http://www.itl.nist.gov/div898/handbook/pmc/section4/pmc435.htm). Maintenant que nous avons la composante saisonnière, nous pouvons prédire non seulement un ou deux pas en avant, mais un futur arbitraire $m$, ce qui est très encourageant.

Vous trouverez ci-dessous le code d'un modèle de lissage exponentiel triple, également connu sous les noms de famille de ses créateurs, Charles Holt et son élève Peter Winters. De plus, la méthode Brutlag a été incluse dans le modèle pour produire des intervalles de confiance:

$$\hat y_{max_x}=\ell_{x−1}+b_{x−1}+s_{x−T}+m⋅d_{t−T}$$

$$\hat y_{min_x}=\ell_{x−1}+b_{x−1}+s_{x−T}-m⋅d_{t−T}$$

$$d_t=\gamma∣y_t−\hat y_t∣+(1−\gamma)d_{t−T},$$

où $T$ est la durée de la saison, $d$ est l'écart prévu. D'autres paramètres ont été tirés du triple lissage exponentiel. Vous pouvez en savoir plus sur la méthode et son applicabilité à la détection d'anomalies dans des séries chronologiques [ici](http://fedcsis.org/proceedings/2012/pliks/118.pdf).

In [ ]:
class HoltWinters:

    """
    Holt-Winters model with the anomalies detection using Brutlag method
    
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """

    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor

    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i + self.slen] - self.series[i]) / self.slen
        return sum / self.slen

    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series) / self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(
                sum(self.series[self.slen * j : self.slen * j + self.slen])
                / float(self.slen)
            )
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += (
                    self.series[self.slen * j + i] - season_averages[j]
                )
            seasonals[i] = sum_of_vals_over_avg / n_seasons
        return seasonals

    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []

        seasonals = self.initial_seasonal_components()

        for i in range(len(self.series) + self.n_preds):
            if i == 0:  # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i % self.slen])

                self.PredictedDeviation.append(0)

                self.UpperBond.append(
                    self.result[0] + self.scaling_factor * self.PredictedDeviation[0]
                )

                self.LowerBond.append(
                    self.result[0] - self.scaling_factor * self.PredictedDeviation[0]
                )
                continue

            if i >= len(self.series):  # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m * trend) + seasonals[i % self.slen])

                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1] * 1.01)

            else:
                val = self.series[i]
                last_smooth, smooth = (
                    smooth,
                    self.alpha * (val - seasonals[i % self.slen])
                    + (1 - self.alpha) * (smooth + trend),
                )
                trend = self.beta * (smooth - last_smooth) + (1 - self.beta) * trend
                seasonals[i % self.slen] = (
                    self.gamma * (val - smooth)
                    + (1 - self.gamma) * seasonals[i % self.slen]
                )
                self.result.append(smooth + trend + seasonals[i % self.slen])

                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(
                    self.gamma * np.abs(self.series[i] - self.result[i])
                    + (1 - self.gamma) * self.PredictedDeviation[-1]
                )

            self.UpperBond.append(
                self.result[-1] + self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.LowerBond.append(
                self.result[-1] - self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i % self.slen])

## Validation croisée des séries chronologiques

Avant de commencer à construire un modèle, voyons d'abord comment estimer automatiquement les paramètres du modèle.

Il n'y a rien d'inhabituel ici; comme toujours, nous devons choisir une fonction de perte adaptée à la tâche qui nous dira à quel point le modèle se rapproche des données. Ensuite, en utilisant la validation croisée, nous évaluerons notre fonction de perte choisie pour les paramètres de modèle donnés, calculerons le gradient, ajusterons les paramètres de modèle, etc., pour finalement descendre au minimum global.

Vous vous demandez peut-être comment effectuer la validation croisée pour les séries temporelles car les séries temporelles ont cette structure temporelle et on ne peut pas mélanger aléatoirement les valeurs dans un pli tout en préservant cette structure. Avec la randomisation, toutes les dépendances temporelles entre les observations seront perdues. C'est pourquoi nous devrons utiliser une approche plus délicate pour optimiser les paramètres du modèle. Je ne sais pas s'il y a un nom officiel à cela, mais sur [CrossValidated](https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model -sélection), où l'on peut trouver toutes les réponses sauf la réponse à la question ultime de la vie, de l'univers et de tout, le nom proposé pour cette méthode est "validation croisée sur une base continue".

L'idée est assez simple - nous formons notre modèle sur un petit segment de la série chronologique du début jusqu'à certains $t$, faisons des prédictions pour les prochaines étapes de $t+n$ et calculons une erreur. Ensuite, nous étendons notre échantillon d'entraînement à la valeur $t+n$, faisons des prédictions de $t+n$ à $t+2*n$ et continuons de déplacer notre segment de test de la série chronologique jusqu'à ce que nous atteignions la dernière observation disponible. Par conséquent, nous avons autant de plis que $n$ se situera entre l'échantillon d'apprentissage initial et la dernière observation.

<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/time_series_cv.png"/>

Maintenant, sachant comment configurer la validation croisée, nous pouvons trouver les paramètres optimaux pour le modèle Holt-Winters. Rappelons que nous avons une saisonnalité quotidienne dans les publicités, d'où le paramètre `slen = 24`.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit  # you have everything done for you


def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=24):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # errors array
    errors = []

    values = series.values
    alpha, beta, gamma = params

    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)

    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):

        model = HoltWinters(
            series=values[train],
            slen=slen,
            alpha=alpha,
            beta=beta,
            gamma=gamma,
            n_preds=len(test),
        )
        model.triple_exponential_smoothing()

        predictions = model.result[-len(test) :]
        actual = values[test]
        error = loss_function(predictions, actual)
        errors.append(error)

    return np.mean(np.array(errors))

Dans le modèle Holt-Winters, ainsi que dans les autres modèles de lissage exponentiel, il existe une contrainte sur la taille des paramètres de lissage, chacun variant de 0 à 1. Par conséquent, afin de minimiser notre fonction de perte, nous devons choisir un algorithme qui prend en charge les contraintes sur les paramètres du modèle. Dans notre cas, nous utiliserons le gradient conjugué de Newton tronqué.

In [ ]:
%%time
data = ads.Ads[:-20]  # leave some data for testing

# initializing model parameters alpha, beta and gamma
x = [0, 0, 0]

# Minimizing the loss function
opt = minimize(
    timeseriesCVscore,
    x0=x,
    args=(data, mean_squared_log_error),
    method="TNC",
    bounds=((0, 1), (0, 1), (0, 1)),
)

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

# ...and train the model with them, forecasting for the next 50 hours
model = HoltWinters(
    data,
    slen=24,
    alpha=alpha_final,
    beta=beta_final,
    gamma=gamma_final,
    n_preds=50,
    scaling_factor=3,
)
model.triple_exponential_smoothing()

Ajoutons du code pour afficher les tracés.

In [ ]:
def plotHoltWinters(series, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """

    plt.figure(figsize=(20, 10))
    plt.plot(model.result, label="Model")
    plt.plot(series.values, label="Actual")
    error = mean_absolute_percentage_error(series.values, model.result[: len(series)])
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))

    if plot_anomalies:
        anomalies = np.array([np.NaN] * len(series))
        anomalies[series.values < model.LowerBond[: len(series)]] = series.values[
            series.values < model.LowerBond[: len(series)]
        ]
        anomalies[series.values > model.UpperBond[: len(series)]] = series.values[
            series.values > model.UpperBond[: len(series)]
        ]
        plt.plot(anomalies, "o", markersize=10, label="Anomalies")

    if plot_intervals:
        plt.plot(model.UpperBond, "r--", alpha=0.5, label="Up/Low confidence")
        plt.plot(model.LowerBond, "r--", alpha=0.5)
        plt.fill_between(
            x=range(0, len(model.result)),
            y1=model.UpperBond,
            y2=model.LowerBond,
            alpha=0.2,
            color="grey",
        )

    plt.vlines(
        len(series),
        ymin=min(model.LowerBond),
        ymax=max(model.UpperBond),
        linestyles="dashed",
    )
    plt.axvspan(len(series) - 20, len(model.result), alpha=0.3, color="lightgrey")
    plt.grid(True)
    plt.axis("tight")
    plt.legend(loc="best", fontsize=13);

In [ ]:
plotHoltWinters(ads.Ads)

In [ ]:
plotHoltWinters(ads.Ads, plot_intervals=True, plot_anomalies=True)

À en juger par les graphiques, notre modèle a réussi à approximer avec succès la série chronologique initiale, capturant la saisonnalité quotidienne, la tendance globale à la baisse et même certaines anomalies. Si vous regardez les écarts du modèle, vous pouvez clairement voir que le modèle réagit assez fortement aux changements dans la structure de la série, puis ramène rapidement l'écart aux valeurs normales, "oubliant" essentiellement le passé. Cette caractéristique du modèle nous permet de construire rapidement des systèmes de détection d'anomalies, même pour les données de série bruyantes, sans dépenser trop de temps et d'argent pour préparer les données et entraîner le modèle.

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis("tight")
plt.title("Brutlag's predicted deviation");

Nous appliquerons le même algorithme pour la deuxième série qui, comme vous vous en souvenez peut-être, a une tendance et une saisonnalité de 30 jours.

In [ ]:
%%time
data = currency.GEMS_GEMS_SPENT[:-50]
slen = 30  # 30-day seasonality

x = [0, 0, 0]

opt = minimize(
    timeseriesCVscore,
    x0=x,
    args=(data, mean_absolute_percentage_error, slen),
    method="TNC",
    bounds=((0, 1), (0, 1), (0, 1)),
)

alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

model = HoltWinters(
    data,
    slen=slen,
    alpha=alpha_final,
    beta=beta_final,
    gamma=gamma_final,
    n_preds=100,
    scaling_factor=3,
)
model.triple_exponential_smoothing()

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT)

Cela semble bon! Le modèle a pris à la fois la tendance à la hausse et les pointes saisonnières et correspond assez bien aux données.

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT, plot_intervals=True, plot_anomalies=True)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis("tight")
plt.title("Brutlag's predicted deviation");

# Approche économétrique

### Stationnarité (Stationarity)

Avant de commencer la modélisation, nous devons mentionner une propriété importante des séries chronologiques: [**stationarity**](https://fr.wikipedia.org/wiki/Processus_stationnaire).

Si un processus est stationnaire, cela signifie qu'il ne change pas ses propriétés statistiques dans le temps, à savoir sa moyenne et sa variance. (La constance de la variance est appelée [homoscédasticité](https://fr.wikipedia.org/wiki/Homosc%C3%A9dasticit%C3%A9)) La fonction de covariance ne dépend pas du temps; cela ne devrait dépendre que de la distance entre les observations. Vous pouvez le voir visuellement sur les images dans le post de [Sean Abu](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/):

- Le graphique rouge ci-dessous n'est pas stationnaire car la moyenne augmente avec le temps.

<img src="https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png"/>

- Nous n'avons pas eu de chance avec la variance et nous voyons la dispersion variable des valeurs dans le temps

<img src="https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png"/>

- Enfin, la covariance du i ème terme et du (i + m) ème terme ne doit pas être fonction du temps. Dans le graphique suivant, vous remarquerez que l'écart se rapproche à mesure que le temps augmente. Par conséquent, la covariance n'est pas constante avec le temps dans le graphique de droite.

<img src="https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png"/>

Alors pourquoi la stationnarité est-elle si importante? Parce qu'il est facile de faire des prédictions sur une série stationnaire car on peut supposer que les futures propriétés statistiques ne seront pas différentes de celles actuellement observées. La plupart des modèles de séries chronologiques, d'une manière ou d'une autre, tentent de prédire ces propriétés (moyenne ou variance, par exemple). De fausses prédictions seraient fausses si la série originale n'était pas stationnaire. Malheureusement, la plupart des séries chronologiques que nous voyons en dehors des manuels scolaires ne sont pas fixes, mais nous pouvons (et devons) changer cela.

Donc, pour lutter contre la non-stationnarité, nous devons pour ainsi dire connaître notre ennemi. Voyons comment nous pouvons le détecter. Nous examinerons le bruit blanc et les promenades aléatoires pour apprendre à passer gratuitement de l'un à l'autre.

Graphique du bruit blanc:

In [ ]:
white_noise = np.random.normal(size=1000)
with plt.style.context("bmh"):
    plt.figure(figsize=(15, 5))
    plt.plot(white_noise)

Le processus généré par la distribution normale standard est stationnaire et oscille autour de 0 avec un écart de 1. Maintenant, sur la base de ce processus, nous allons générer un nouveau où chaque valeur suivante dépendra de la précédente: $x_t = \rho x_{t-1} + e_t$

Voici le code pour afficher les tracés.

In [ ]:
def plotProcess(n_samples=1000, rho=0):
    x = w = np.random.normal(size=n_samples)
    for t in range(n_samples):
        x[t] = rho * x[t - 1] + w[t]

    with plt.style.context("bmh"):
        plt.figure(figsize=(10, 3))
        plt.plot(x)
        plt.title(
            "Rho {}\n Dickey-Fuller p-value: {}".format(
                rho, round(sm.tsa.stattools.adfuller(x)[1], 3)
            )
        )


for rho in [0, 0.6, 0.9, 1]:
    plotProcess(rho=rho)

Sur le premier tracé, vous pouvez voir le même bruit blanc stationnaire qu'auparavant. Sur le deuxième graphique avec $\rho$ augmenté à 0,6, des cycles plus larges sont apparus, mais il semble toujours stationnaire dans l'ensemble. Le troisième graphique s'écarte encore plus de la moyenne de 0 mais oscille toujours autour de la moyenne. Enfin, avec $\rho=1$, nous avons un processus de marche aléatoire, c'est-à-dire une série chronologique non stationnaire.

Cela se produit car, après avoir atteint la valeur critique, la série $x_t = \rho x_{t-1} + e_t$ ne revient pas à sa valeur moyenne. Si nous soustrayons $x_{t-1}$ des deux côtés, nous obtiendrons $x_t - x_{t-1} = (\rho - 1) x_{t-1} + e_t$, où l'expression de gauche est appelée la première différence. Si $\rho=1$, alors la première différence nous donne $e_t$ de bruit blanc stationnaire. C'est l'idée principale derrière le [test de Dickey-Fuller](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) pour la stationnarité des séries chronologiques (test de la présence d'une racine unitaire). Si nous pouvons obtenir une série stationnaire à partir d'une série non stationnaire en utilisant la première différence, nous appelons ces séries intégrées d'ordre 1. L'hypothèse nulle du test est que la série chronologique est non stationnaire, ce qui a été rejeté sur les trois premiers graphiques et finalement accepté sur le dernier. Nous devons dire que la première différence n'est pas toujours suffisante pour obtenir une série stationnaire car le processus peut être intégré d'ordre d, d> 1 (et avoir plusieurs racines unitaires). Dans ce cas, on utilise le test Dickey-Fuller amélioré, qui vérifie plusieurs retards à la fois

Nous pouvons lutter contre la non-stationnarité en utilisant différentes approches: diverses différences d'ordre, suppression de tendance et de saisonnalité, lissage et transformations comme Box-Cox ou logarithmique.

## Se débarrasser de la non-stationnarité et créer SARIMA

Construisons un modèle ARIMA en parcourant tous les étapes de création d'une série stationnaire.

Voici le code pour afficher les tracés.

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style="bmh"):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test
        
        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title(
            "Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}".format(p_value)
        )
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(ads.Ads, lags=60)

_this outlier on partial autocorrelation plot looks like a statsmodels bug, partial autocorrelation shall be <= 1 like any correlation._

Surprisingly, the initial series are stationary; the Dickey-Fuller test rejected the null hypothesis that a unit root is present. Actually, we can see this on the plot itself – we do not have a visible trend, so the mean is constant and the variance is pretty much stable. The only thing left is seasonality, which we have to deal with prior to modeling. To do so, let's take the "seasonal difference", which means a simple subtraction of the series from itself with a lag that equals the seasonal period.

_cette valeur aberrante sur le tracé d'autocorrélation partielle ressemble à un bug des modèles statistiques, l'autocorrélation partielle doit être <= 1 comme toute corrélation._

Étonnamment, les premières séries sont stationnaires; le test de Dickey-Fuller a rejeté l'hypothèse nulle selon laquelle une racine unitaire est présente. En fait, nous pouvons le voir sur le graphique lui-même - nous n'avons pas de tendance visible, donc la moyenne est constante et la variance est à peu près stable. La seule chose qui reste est la saisonnalité, à laquelle nous devons faire face avant la modélisation. Pour ce faire, prenons la «différence saisonnière», ce qui signifie une simple soustraction de la série d'elle-même avec un décalage égal à la période saisonnière.

In [ ]:
ads_diff = ads.Ads - ads.Ads.shift(24)
tsplot(ads_diff[24:], lags=60)

C'est maintenant beaucoup mieux avec la saisonnalité visible disparue. Cependant, la fonction d'autocorrélation a encore trop de retards importants. Pour les supprimer, nous prendrons les premières différences, en soustrayant la série d'elle-même avec le décalage 1.

In [ ]:
ads_diff = ads_diff - ads_diff.shift(1)
tsplot(ads_diff[24 + 1 :], lags=60)

Parfait! Notre série ressemble maintenant à quelque chose d'indescriptible, oscillant autour de zéro. Le test de Dickey-Fuller indique qu'il est stationnaire et le nombre de pics significatifs dans ACF a chuté. Nous pouvons enfin commencer la modélisation!

## Cours intensif sur les méthodes ARIMA

Nous expliquerons ce modèle en construisant lettre par lettre. $SARIMA(p, d, q)(P, D, Q, s)$, (Seasonal Autoregression Moving Average model) modèle de moyenne mobile d'autorégression saisonnière:

- $AR(p)$ - modèle d'autorégression, c'est-à-dire régression de la série chronologique sur elle-même. L'hypothèse de base est que les valeurs actuelles de la série dépendent de ses valeurs précédentes avec un certain décalage (ou plusieurs décalages). Le décalage maximum dans le modèle est appelé $p$. Pour déterminer le $p$ initial, vous devez regarder le tracé PACF et trouver le plus grand décalage significatif après lequel **la plupart** des autres décalages deviennent insignifiants.
- $MA(q)$ - modèle de moyenne mobile. Sans entrer dans trop de détails, cela modélise l'erreur de la série chronologique, là encore avec l'hypothèse que l'erreur actuelle dépend de la précédente avec un certain retard, ce qui est appelé $q$. La valeur initiale peut être trouvée sur le tracé ACF avec la même logique que précédemment.

Combinons nos 4 premières lettres:

$AR(p) + MA(q) = ARMA(p, q)$

Ce que nous avons ici, c'est le modèle autorégressif à moyenne mobile! Si la série est stationnaire, elle peut être approximée avec ces 4 lettres. Nous allons continuer.

- $I(d)$ - ordre d'intégration. Il s'agit simplement du nombre de différences non saisonnières nécessaires pour rendre la série stationnaire. Dans notre cas, c'est juste 1 car nous avons utilisé les premières différences.

L'ajout de cette lettre aux quatre autres nous donne le modèle $ARIMA$ qui peut gérer des données non stationnaires à l'aide de différences non saisonnières. Super, une lettre de plus à faire!

- $S(s)$ - est responsable de la saisonnalité et est égal à la durée de la saison de la série

Avec cela, nous avons trois paramètres: $(P, D, Q)$

- $P$ - ordre d'autorégression pour la composante saisonnière du modèle, qui peut être dérivé du PACF. Mais vous devez regarder le nombre de retards importants, qui sont les multiples de la durée de la saison. Par exemple, si la période est égale à 24 et que les décalages des 24e et 48e sont significatifs dans le PACF, cela signifie que le $P$ initial devrait être 2.

- $Q$ - logique similaire utilisant le graphique ACF à la place.

- $D$ - ordre d'intégration saisonnière. Peut être égal à 1 ou 0, selon que des différences saisonnières ont été appliquées ou non.

Maintenant que nous savons comment définir les paramètres initiaux, regardons à nouveau le tracé final et définissons les paramètres:

In [ ]:
tsplot(ads_diff[24 + 1 :], lags=60)

- $p$ est très probablement 4 puisqu'il s'agit du dernier retard significatif sur le PACF, après quoi la plupart des autres ne sont pas significatifs.
- $d$ est égal à 1 car nous avons eu les premières différences
- $q$ devrait se situer aux alentours de 4 ainsi que sur l'ACF
- $P$ pourrait être égal à 2, car les décalages des 24e et 48e sont quelque peu significatifs sur le PACF
- $D$ est à nouveau égal à 1 car nous avons effectué une différenciation saisonnière
- $Q$ vaut probablement 1. Le 24 e décalage sur ACF est significatif tandis que le 48 e ne l'est pas.

Testons différents modèles et voyons lequel est le meilleur.

In [ ]:
# setting initial values and some bounds for them
ps = range(2, 5)
d = 1
qs = range(2, 5)
Ps = range(0, 2)
D = 1
Qs = range(0, 2)
s = 24  # season length is still 24

# creating list with all the possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimizeSARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order 
        s - length of season
    """

    results = []
    best_aic = float("inf")

    for param in tqdm_notebook(parameters_list):
        # we need try-except because on some combinations model fails to converge
        try:
            model = sm.tsa.statespace.SARIMAX(
                ads.Ads,
                order=(param[0], d, param[1]),
                seasonal_order=(param[2], D, param[3], s),
            ).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ["parameters", "aic"]
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by="aic", ascending=True).reset_index(
        drop=True
    )

    return result_table

In [ ]:
%%time
result_table = optimizeSARIMA(parameters_list, d, D, s)

In [ ]:
result_table.head()

In [ ]:
# set the parameters that give the lowest AIC
p, q, P, Q = result_table.parameters[0]

best_model = sm.tsa.statespace.SARIMAX(
    ads.Ads, order=(p, d, q), seasonal_order=(P, D, Q, s)
).fit(disp=-1)
print(best_model.summary())

Let's inspect the residuals of the model.

In [ ]:
tsplot(best_model.resid[24 + 1 :], lags=60)

Il est clair que les résidus sont stationnaires et il n'y a aucune autocorrélation apparente. Faisons des prédictions en utilisant notre modèle.

In [ ]:
def plotSARIMA(series, model, n_steps):
    """
        Plots model vs predicted values
        
        series - dataset with timeseries
        model - fitted SARIMA model
        n_steps - number of steps to predict in the future
        
    """
    # adding model values
    data = series.copy()
    data.columns = ["actual"]
    data["arima_model"] = model.fittedvalues
    # making a shift on s+d steps, because these values were unobserved by the model
    # due to the differentiating
    data["arima_model"][: s + d] = np.NaN

    # forecasting on n_steps forward
    forecast = model.predict(start=data.shape[0], end=data.shape[0] + n_steps)
    forecast = data.arima_model.append(forecast)
    # calculate error, again having shifted on s+d steps from the beginning
    error = mean_absolute_percentage_error(
        data["actual"][s + d :], data["arima_model"][s + d :]
    )

    plt.figure(figsize=(15, 7))
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    plt.plot(forecast, color="r", label="model")
    plt.axvspan(data.index[-1], forecast.index[-1], alpha=0.5, color="lightgrey")
    plt.plot(data.actual, label="actual")
    plt.legend()
    plt.grid(True);

In [ ]:
plotSARIMA(ads, best_model, 50)

Au final, nous avons obtenu des prévisions très adéquates. Notre modèle s'est trompé de 4,01% en moyenne, ce qui est très, très bon. Cependant, les coûts globaux de préparation des données, de mise à l'arrêt de la série et de sélection des paramètres peuvent ne pas valoir cette précision.

# Modèles linéaires (et pas tout à fait) pour les séries chronologiques

Souvent, dans mon travail, je dois construire des modèles [*rapide, bon, bon marché*](http://fastgood.cheap) comme seul principe directeur. Cela signifie que certains de ces modèles ne seront jamais considérés comme «prêts pour la production» car ils demandent trop de temps pour la préparation des données (comme dans SARIMA) ou nécessitent un recyclage fréquent sur les nouvelles données (encore une fois, SARIMA) ou sont difficiles à régler (bon exemple - SARIMA). Par conséquent, il est très souvent beaucoup plus facile de sélectionner quelques entités à partir des séries chronologiques existantes et de créer un modèle de régression linéaire simple ou, par exemple, une forêt aléatoire. C'est bon et pas cher.

Cette approche n'est pas soutenue par la théorie et casse plusieurs hypothèses (par exemple le théorème de Gauss-Markov, en particulier pour les erreurs non corrélées), mais elle est très utile dans la pratique et est souvent utilisée dans les compétitions d'apprentissage automatique.

## Extraction de caractéristiques

Le modèle a besoin de caractéristiques, et tout ce que nous avons est une série chronologique à une dimension. Quelles caractéristiques pouvons-nous extraire?

* Time series lags (Les décalages des séries chronologiques)
* Statistiques de fenêtre:
    - Valeur max / min de la série dans une fenêtre
    - Valeur moyenne / médiane dans une fenêtre
    - Variance de fenêtre
    - etc.
*Caractéristiques de date et d'heure:
    - Minute d'une heure, heure d'un jour, jour de la semaine, etc.
    - Ce jour est-il un jour férié? Peut-être qu'il y a un événement spécial? Représentez cela comme une fonction booléenne
* Encodage cible
* Prévisions d'autres modèles (notez que nous pouvons perdre la vitesse de prédiction de cette façon)

Passons en revue certaines des méthodes et voyons ce que nous pouvons extraire de nos données de séries chronologiques sur les annonces.

## Retards dans les séries chronologiques (Time series lags)

En décalant les pas de la série $n$, nous obtenons une colonne d'entités où la valeur actuelle de la série chronologique est alignée avec sa valeur au moment $t-n$. Si nous effectuons un décalage de 1 et formons un modèle sur cette caractéristique, le modèle sera en mesure de prévoir une longueur d'avance sur l'observation de l'état actuel de la série. Augmenter le décalage, disons jusqu'à 6, permettra au modèle de faire des prévisions 6 pas en avant; cependant, il utilisera les données observées 6 pas en arrière. Si quelque chose change fondamentalement la série au cours de cette période non observée, le modèle n'acceptera pas ces changements et renverra des prévisions avec une grande erreur. Par conséquent, lors de la sélection initiale du décalage, il faut trouver un équilibre entre la qualité de prédiction optimale et la longueur de l'horizon de prévision.

In [ ]:
# Creating a copy of the initial datagrame to make various transformations
data = pd.DataFrame(ads.Ads.copy())
data.columns = ["y"]

In [ ]:
# Adding the lag of the target variable from 6 steps back up to 24
for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

In [ ]:
# take a look at the new dataframe
data.tail(7)

Très bien, nous avons généré un ensemble de données ici. Pourquoi ne formons-nous pas maintenant un modèle?

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# for time-series cross-validation set 5 folds
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """

    # get the index after which test set starts
    test_index = int(len(X) * (1 - test_size))

    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]

    return X_train, X_test, y_train, y_test

In [ ]:
y = data.dropna().y
X = data.dropna().drop(["y"], axis=1)

# reserve 30% of data for testing
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

In [ ]:
# machine learning in two lines
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
def plotModelResults(
    model, X_train=X_train, X_test=X_test, plot_intervals=False, plot_anomalies=False
):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """

    prediction = model.predict(X_test)

    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)

    if plot_intervals:
        cv = cross_val_score(
            model, X_train, y_train, cv=tscv, scoring="neg_mean_absolute_error"
        )
        mae = cv.mean() * (-1)
        deviation = cv.std()

        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)

        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)

        if plot_anomalies:
            anomalies = np.array([np.NaN] * len(y_test))
            anomalies[y_test < lower] = y_test[y_test < lower]
            anomalies[y_test > upper] = y_test[y_test > upper]
            plt.plot(anomalies, "o", markersize=10, label="Anomalies")

    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True)


def plotCoefficients(model):
    """
        Plots sorted coefficient values of the model
    """

    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind="bar")
    plt.grid(True, axis="y")
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles="dashed");

In [ ]:
plotModelResults(lr, plot_intervals=True)
plotCoefficients(lr)

Des retards simples et une régression linéaire nous ont donné des prédictions qui ne sont pas si éloignées de SARIMA en termes de qualité. Il existe de nombreuses caractéristiques inutiles, nous ferons donc la sélection des caractéristiques dans un petit moment. Pour l'instant, continuons l'ingénierie!

Nous ajouterons l'heure, le jour de la semaine et un booléen pour `is_weekend`. Pour ce faire, nous devons transformer l'index actuel de la trame de données au format `datetime` et extraire` hour` et `week`.

In [ ]:
data.index = pd.to_datetime(data.index)
data["hour"] = data.index.hour
data["weekday"] = data.index.weekday
data["is_weekend"] = data.weekday.isin([5, 6]) * 1
data.tail()

Nous pouvons visualiser les caractéristiques résultantes.

In [ ]:
plt.figure(figsize=(16, 5))
plt.title("Encoded features")
data.hour.plot()
data.weekday.plot()
data.is_weekend.plot()
plt.grid(True);

Étant donné que nous avons maintenant différentes échelles dans nos variables, des milliers pour les caractéristiques de décalage et des dizaines pour les caractéristiques catégorielles, nous devons les transformer en la même échelle pour explorer l'importance des caractéristiques et, plus tard, la régularisation.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
y = data.dropna().y
X = data.dropna().drop(["y"], axis=1)

X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelResults(lr, X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True)
plotCoefficients(lr)

L'erreur de test diminue un peu. A en juger par le tracé des coefficients, nous pouvons dire que `week` et` is_weekend` sont des caractéristiques utiles.

## Encodage de la  cible
Je voudrais ajouter une autre variante pour l'encodage des variables catégorielles: l'encodage par valeur moyenne. S'il n'est pas souhaitable d'exploser un ensemble de données en utilisant de nombreuses variables factices qui peuvent entraîner la perte d'informations et si elles ne peuvent pas être utilisées comme valeurs réelles en raison de conflits comme "0 heures <23 heures", il est alors possible de coder une variable avec des valeurs légèrement plus interprétables. L'idée naturelle est de coder avec la valeur moyenne de la variable cible. Dans notre exemple, chaque jour de la semaine et chaque heure de la journée peut être codé par le nombre moyen correspondant d'annonces vues au cours de cette journée ou de cette heure. Il est très important de s'assurer que la valeur moyenne est calculée sur l'ensemble d'apprentissage uniquement (ou sur le 'pli' (fold) de validation croisée actuel uniquement) afin que le modèle ne soit pas au courant de l'avenir.

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Returns a dictionary where keys are unique categories of the cat_feature,
    and values are means over real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

Regardons les moyennes par heure.

In [ ]:
average_hour = code_mean(data, "hour", "y")
plt.figure(figsize=(7, 5))
plt.title("Hour averages")
pd.DataFrame.from_dict(average_hour, orient="index")[0].plot()
plt.grid(True);

Enfin, rassemblons toutes les transformations dans une seule fonction.

In [ ]:
def prepareData(series, lag_start, lag_end, test_size, target_encoding=False):
    """
        series: pd.DataFrame
            dataframe with timeseries

        lag_start: int
            initial step back in time to slice target variable 
            example - lag_start = 1 means that the model 
                      will see yesterday's values to predict today

        lag_end: int
            final step back in time to slice target variable
            example - lag_end = 4 means that the model 
                      will see up to 4 days back in time to predict today

        test_size: float
            size of the test dataset after train/test split as percentage of dataset

        target_encoding: boolean
            if True - add target averages to the dataset
        
    """

    # copy of the initial dataset
    data = pd.DataFrame(series.copy())
    data.columns = ["y"]

    # lags of series
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.y.shift(i)

    # datetime features
    data.index = pd.to_datetime(data.index)
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data["is_weekend"] = data.weekday.isin([5, 6]) * 1

    if target_encoding:
        # calculate averages on train set only
        test_index = int(len(data.dropna()) * (1 - test_size))
        data["weekday_average"] = list(
            map(code_mean(data[:test_index], "weekday", "y").get, data.weekday)
        )
        data["hour_average"] = list(
            map(code_mean(data[:test_index], "hour", "y").get, data.hour)
        )

        # drop encoded variables
        data.drop(["hour", "weekday"], axis=1, inplace=True)

    # train-test split
    y = data.dropna().y
    X = data.dropna().drop(["y"], axis=1)
    X_train, X_test, y_train, y_test = timeseries_train_test_split(
        X, y, test_size=test_size
    )

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = prepareData(
    ads.Ads, lag_start=6, lag_end=25, test_size=0.3, target_encoding=True
)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelResults(
    lr,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(lr)

Nous voyons un **surajustement**! `Hour_average` était si bien dans l'ensemble de données d'entraînement que le modèle a décidé de concentrer toutes ses forces sur celui-ci. En conséquence, la qualité de la prévision a chuté. Ce problème peut être résolu de différentes manières; par exemple, nous pouvons calculer l'encodage cible non pas pour l'ensemble du train, mais pour une certaine fenêtre à la place. De cette façon, les encodages de la dernière fenêtre observée décrivent probablement mieux l'état actuel de la série. Alternativement, nous pouvons simplement le supprimer manuellement car nous sommes sûrs que cela ne fait qu'empirer les choses dans ce cas.

In [ ]:
X_train, X_test, y_train, y_test = prepareData(
    ads.Ads, lag_start=6, lag_end=25, test_size=0.3, target_encoding=False
)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Régularisation et sélection des caractéristiques

Comme nous le savons déjà, toutes les fonctionnalités ne sont pas également saines - certaines peuvent entraîner un sur-ajustement tandis que d'autres doivent être supprimées. Outre l'inspection manuelle, nous pouvons appliquer la régularisation. Les régressions Ridge et Lasso sont deux des modèles de régression les plus populaires avec régularisation. Ils ajoutent tous deux des contraintes supplémentaires à notre fonction de perte.

Dans le cas de la régression Ridge, ces contraintes sont la somme des carrés des coefficients multipliés par le coefficient de régularisation. Plus le coefficient d'une caractéristique est élevé, plus notre perte sera importante. Par conséquent, nous essaierons d'optimiser le modèle tout en maintenant les coefficients assez bas.

À la suite de cette régularisation $L2$, nous aurons un biais plus élevé et une variance plus faible, donc le modèle généralisera mieux (du moins c'est ce que nous espérons arriver).

Le deuxième modèle de régression, la régression Lasso, ajoute à la fonction de perte, non des carrés, mais des valeurs absolues des coefficients. Par conséquent, au cours du processus d'optimisation, les coefficients des entités non importantes peuvent devenir des zéros, ce qui permet une sélection automatisée des entités. Ce type de régularisation est appelé $L1$.

Tout d'abord, assurons-nous que nous avons des caractéristiques à supprimer et que les données ont des caractéristiques hautement corrélées.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(X_train.corr());

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV

ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)

plotModelResults(
    ridge,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(ridge)

Nous pouvons clairement voir que certains coefficients se rapprochent de plus en plus de zéro (bien qu'ils ne l'atteignent jamais réellement) à mesure que leur importance dans le modèle diminue.

In [ ]:
lasso = LassoCV(cv=tscv)
lasso.fit(X_train_scaled, y_train)

plotModelResults(
    lasso,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(lasso)

La régression Lasso s'est avérée plus conservatrice; il a supprimé le 23 e décalage des caractéristiques les plus importantes et a complètement supprimé 5 caractéristiques, ce qui n'a fait qu'améliorer la qualité des prévisions.

# Boosting 
Why shouldn't we try XGBoost now?
<img src="../../img/xgboost_the_things.jpg"/>

# Boosting
Pourquoi ne devrions-nous pas essayer XGBoost maintenant?
<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/xgboost-all-the-things.jpg"/>

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(verbosity=0)
xgb.fit(X_train_scaled, y_train);

In [ ]:
plotModelResults(
    xgb,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)

Nous avons un gagnant! Il s'agit de la plus petite erreur sur l'ensemble de test parmi tous les modèles que nous avons essayés jusqu'à présent.

Mais, cette victoire est trompeuse, et ce n'est peut-être pas l'idée la plus brillante d'intégrer `xgboost` dès que vous obtenez les données de séries chronologiques. En règle générale, les modèles basés sur les arbres gèrent mal les tendances des données par rapport aux modèles linéaires. Dans ce cas, vous devrez d'abord détruire votre série ou utiliser des astuces pour que la magie se produise. Idéalement, vous pouvez rendre la série stationnaire, puis utiliser XGBoost. Par exemple, vous pouvez prévoir la tendance séparément avec un modèle linéaire, puis ajouter des prévisions à partir de `xgboost` pour obtenir une prévision finale.

# Conclusion

Nous avons discuté de différentes méthodes d'analyse et de prévision des séries chronologiques. Malheureusement, ou peut-être heureusement, il n'y a pas une seule façon de résoudre ce genre de problèmes. Les méthodes développées dans les années 1960 (et certaines même au début du XXIe siècle) sont toujours populaires, tout comme les LSTM et les RNN (non couvertes dans cet article). Cela est partiellement lié au fait que la tâche de prédiction, comme toute autre tâche liée aux données, nécessite de la créativité à bien des égards et nécessite certainement des recherches. Malgré le grand nombre de métriques de qualité formelles et d'approches d'estimation des paramètres, il est souvent nécessaire d'essayer quelque chose de différent pour chaque série chronologique. Enfin et surtout, l'équilibre entre qualité et coût est important. À titre d'exemple, le modèle SARIMA peut produire des résultats spectaculaires après réglage, mais peut nécessiter de nombreuses heures de ~~danse du tambourin~~ manipulation de séries temporelles tandis qu'un modèle de régression linéaire simple peut être construit en 10 minutes et peut obtenir des résultats plus ou moins comparables.

# Ressources utiles

* The same notebook as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-9-part-1-time-series-analysis-in-python)
* "LSTM (Long Short Term Memory) Networks for predicting Time Series" - a tutorial by Max Sergei Bulaev within mlcourse.ai (full list of tutorials is [here](https://mlcourse.ai/tutorials))
* Main course [site](https://mlcourse.ai), [course repo](https://github.com/Yorko/mlcourse.ai), and YouTube [channel](https://www.youtube.com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX)
* Course materials as a [Kaggle Dataset](https://www.kaggle.com/kashnitsky/mlcourse)
* Medium ["story"](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3?source=collection_home---6------2---------------------) based on this notebook
* If you read Russian: an [article](https://habr.com/ru/company/ods/blog/327242/) on Habr.com with ~ the same material. And a [lecture](https://youtu.be/_9lBwXnbOd8) on YouTube
* [Online textbook](https://people.duke.edu/~rnau/411home.htm) for the advanced statistical forecasting course at Duke University - covers various smoothing techniques in detail along with linear and ARIMA models
* [Comparison of ARIMA and Random Forest time series models for prediction of avian influenza H5N1 outbreaks](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-276) - one of a few cases where using random forest for time series forecasting is actively defended
* [Time Series Analysis (TSA) in Python - Linear Models to GARCH](http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016) - applying the ARIMA models family to the task of modeling financial indicators (by Brian Christopher)Christopher)